In [ ]:
!mkdir -p sample_data
!gsutil -m cp "gs://applepen-input-1/AK-b2-inventory.csv" sample_data/

In [1]:
import pandas as pd

In [3]:
df_ak_inventory = pd.read_csv("sample_data/AK-b2-inventory.csv")

In [4]:
df_ak_inventory.sort_values("date")

,date,apple,pen
0,2006-01-31,11861,868
1,2006-02-28,30460,2300
2,2006-03-31,42056,3285
3,2006-04-30,55428,4304
4,2006-05-31,67787,5218
...,...,...,...
115,2015-08-31,1522550,119778
116,2015-09-30,1536506,120885
117,2015-10-31,1549174,121760
118,2015-11-30,1563136,122757


In [5]:
df_ak_supply = pd.read_csv("sample_data/AK-b2-supply.csv")

In [6]:
df_ak_supply.sort_values("date")

,date,apple,pen
0,2006-01-01,34949,2680
1,2006-01-15,35069,2651
2,2006-02-01,35210,2724
3,2006-02-15,35157,2748
4,2006-03-01,34960,2718
...,...,...,...
235,2015-10-15,34928,2655
236,2015-11-01,34937,2695
237,2015-11-15,35169,2659
238,2015-12-01,35144,2709


In [7]:
sell_df = pd.read_csv("sample_data/AK-b2-sell.csv")

In [8]:
sell_df.sort_values("date")

,date,sku_num
0,2006-01-01,AK-b2-ap-a7669566-6899-4dff-a1d6-842ad013ee84
1385,2006-01-01,AK-b2-ap-ab812054-bf3f-4f01-845e-0f6999986939
1384,2006-01-01,AK-b2-ap-6dd86f68-d3f7-4928-b1ce-829a5d525f16
1383,2006-01-01,AK-b2-ap-14cc02ed-4cc6-4f08-a3f7-0bbf44e07094
1382,2006-01-01,AK-b2-ap-be3be8b5-1f8f-4c15-82c0-81fd591bdb96
...,...,...
7346189,2015-12-31,AK-b2-ap-a7cd03d9-be7f-409a-b2de-b8497759be7e
7346188,2015-12-31,AK-b2-ap-6140aeed-fd14-48b1-84fa-4b2a08d00059
7346187,2015-12-31,AK-b2-ap-2a0f4613-2982-4511-bd28-d6ebc907329c
7346200,2015-12-31,AK-b2-ap-0d45146f-e5eb-4cbd-9533-35dadb5c47fb


In [9]:
products = {
    'ap': 'apple',
    'pe': 'pen'
}

def to_product(sku):
    return products[sku.split("-")[2]]

sell_df['product'] = sell_df['sku_num'].apply(to_product)

In [10]:
pivot = sell_df \
    .groupby(['date', 'product'])['sku_num'] \
    .count() \
    .reset_index() \
    .rename(columns={'sku_num': 'sales'}) \
    .pivot(index='date', columns='product')

pivot

sales     
product    apple  pen
date                 
2006-01-01  1922  143
2006-01-02  1819  158
2006-01-03  1815  145
2006-01-04  1923  152
2006-01-05  1900  131
...          ...  ...
2015-12-27  1892  153
2015-12-28  1854  153
2015-12-29  1830  150
2015-12-30  1817  151
2015-12-31  1845  140

[3652 rows x 2 columns]

In [11]:
df = pd.concat([
    pd.merge(pivot.sales, df_ak_inventory, on='date', how='left', suffixes=('_sold', '_inventory')),
    pd.merge(pivot.sales, df_ak_supply, on='date', how='left', suffixes=('_sold', '_supply'))[['apple_supply', 'pen_supply']]
], axis=1)

In [12]:
df

,date,apple_sold,pen_sold,apple_inventory,pen_inventory,apple_supply,pen_supply
0,2006-01-01,1922,143,NaN,NaN,34949.0,2680.0
1,2006-01-02,1819,158,NaN,NaN,NaN,NaN
2,2006-01-03,1815,145,NaN,NaN,NaN,NaN
3,2006-01-04,1923,152,NaN,NaN,NaN,NaN
4,2006-01-05,1900,131,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3647,2015-12-27,1892,153,NaN,NaN,NaN,NaN
3648,2015-12-28,1854,153,NaN,NaN,NaN,NaN
3649,2015-12-29,1830,150,NaN,NaN,NaN,NaN
3650,2015-12-30,1817,151,NaN,NaN,NaN,NaN


In [13]:
s = pd.Series(range(5))
s

0    0
1    1
2    2
3    3
4    4
dtype: int64

In [17]:
df['expected_pen_inventory'] = (df.pen_supply.fillna(0).cumsum() - df.pen_sold.cumsum()).astype(int)
df['expected_apple_inventory'] = (df.apple_supply.fillna(0).cumsum() - df.apple_sold.cumsum()).astype(int)

In [18]:
df

,date,apple_sold,pen_sold,apple_inventory,pen_inventory,apple_supply,pen_supply,expected_pen_inventory,expected_apple_inventory
0,2006-01-01,1922,143,NaN,NaN,34949.0,2680.0,2537,33027
1,2006-01-02,1819,158,NaN,NaN,NaN,NaN,2379,31208
2,2006-01-03,1815,145,NaN,NaN,NaN,NaN,2234,29393
3,2006-01-04,1923,152,NaN,NaN,NaN,NaN,2082,27470
4,2006-01-05,1900,131,NaN,NaN,NaN,NaN,1951,25570
...,...,...,...,...,...,...,...,...,...
3647,2015-12-27,1892,153,NaN,NaN,NaN,NaN,125224,1583669
3648,2015-12-28,1854,153,NaN,NaN,NaN,NaN,125071,1581815
3649,2015-12-29,1830,150,NaN,NaN,NaN,NaN,124921,1579985
3650,2015-12-30,1817,151,NaN,NaN,NaN,NaN,124770,1578168


# 1. Daily inventory for each store

In [16]:
df[['date', 'expected_apple_inventory', 'expected_pen_inventory']] \
    .set_index('date') \
    .rename(columns={'expected_apple_inventory': 'apple', 'expected_pen_inventory': 'pen'}) \
    .to_csv('sample_data/daily_inventory.csv')

SyntaxError: unexpected character after line continuation character (396405200.py, line 3)

In [ ]:
compare_df = df[['date', 'apple_inventory', 'pen_inventory', 'expected_pen_inventory', 'expected_apple_inventory']].dropna()

In [ ]:
compare_df['apple_stolen'] = (compare_df['expected_apple_inventory'] - compare_df['apple_inventory']).astype(int)
compare_df['pen_stolen'] = (compare_df['expected_pen_inventory'] - compare_df['pen_inventory']).astype(int)

In [ ]:
compare_df

# 2. Monthly amount of stolen goods for each store.

In [ ]:
compare_df[['date', 'apple_stolen', 'pen_stolen']] \
    .set_index('date') \
    .rename(columns={"apple_stolen": 'apple', 'pen_stolen': 'pen'}) \
    .to_csv("sample_data/monthly_stolen.csv")

In [ ]:
compare_df['year'] = pd.to_datetime(compare_df['date']).dt.year

In [ ]:
compare_df.groupby('year')[['apple_stolen', 'pen_stolen']].sum()

In [ ]:
df['year'] = pd.to_datetime(df['date']).dt.year

In [ ]:
df.groupby('year')[['apple_sold', 'pen_sold']].sum()

In [ ]:
overview_df = pd.concat([
    compare_df.groupby('year')[['apple_stolen', 'pen_stolen']].sum(),
    df.groupby('year')[['apple_sold', 'pen_sold']].sum()
], axis=1)

In [ ]:
overview_df['state'] = 'AK'

# 3. High level overview with sales volume and amount of stolen goods by state and year.

In [ ]:
overview_df[['state', 'apple_sold', 'apple_stolen', 'pen_sold', 'pen_stolen']]

In [ ]:
pd.concat([overview_df, overview_df]).groupby(['year', 'state']).sum().apple_sold.astype('int')

In [ ]:
overview_df.to_csv('sample_data/yearly_overview.csv')

In [10]:
df = pd.read_csv("output/yearly_overview.csv", keep_default_na=False)

In [11]:
df[df.isnull().any(axis=1)]

,year,state,apple_sold,apple_stolen,pen_sold,pen_stolen


In [7]:
df2 = pd.DataFrame({"x": ["NA"]})
df2[df2.isnull().any(axis=1)]

,x
